# Hands On Data in Astronomy Pt. 1

In this first part we will lear how to work with Units, Coordinates and Tables (including fits files)

## Table of contents
 - [0. Setup](#0.-Setup)
 - [1. Units and Quantities](#1.-Units-and-Quantities)
 - [2. Coordinates](#2.-Coordinates)
    - [2.1 ALT / AZ coordinates](#2.1-ALT-/-AZ-coordinates)
 - [3. Tables](#3.-Tables)
    - [3.1 Basics](#3.1-Basics)
    - [3.2 Accessing rows and columns](#3.2-Accessing-rows-and-columns)
    - [3.3 Reading / Writing tables to disk](#3.3-Reading-/-Writing-tables-to-disk)
 - [4. Read FITS files](#4.-Read-FITS-files)
    - [4.1 Primary](#4.1-Primary)
    - [4.2 Events](#4.2-Events)
    - [4.3 GTI](#4.3-GTI)
    - [4.4 Effective Area](#4.4-Effective-Area)
    - [4.5 Energy Dispersion](#4.5-Energy-Dispersion)



## What is Astropy?


    "The Astropy Project is a community effort to develop a single core package for Astronomy in Python and foster interoperability between Python astronomy packages."


The concept and structure of the package is decribed in more detail in the first [Astropy paper 2013](http://adsabs.harvard.edu/abs/2013A%26A...558A..33A). The development infrastructure
and status of the v2.0 core package is described in the second [Astropy paper 2018](http://adsabs.harvard.edu/abs/2018AJ....156..123A).

The **Astropy package is structured into several submodules** and we will cover (what we consider) the most important of them in the following order:

1. [astropy.units](http://docs.astropy.org/en/stable/units/index.html) and in particular [astropy.units.Quantity](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html) to do astronomical calculations with units.

2. [astropy.coordinates](http://docs.astropy.org/en/stable/coordinates/) and in particular the classes [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) and [Angle](http://docs.astropy.org/en/stable/coordinates/angles.html) to handle astronomical sky positions, coordinate systems and coordinate transformations.

3. [astropy.tables](http://docs.astropy.org/en/stable/table/index.html) and the [Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html) class to handle astronomical data tables.

4. [astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html) to open and write data files in [FITS format](https://fits.gsfc.nasa.gov/fits_documentation.html).


# 0. Setup
[back to top](#Table-of-contents)

Check package versions. All examples should work with Astropy > 2.0 and Numpy > 1.11

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import astropy
print('numpy:', np.__version__)
print('astropy:', astropy.__version__)

# 1. Units and Quantities

The [astropy.units]() subpackage provides functions and classes to handle physical quantities with units. 


The recommended way to import the `astropy.units` submodule is: 

In [ ]:
from astropy import units as u

`Quantities` are created by multiplying any number with a unit object:

In [ ]:
distance = 1. * u.parsec
print( distance.to( u.lightyear) )
print(distance)

Or by passing a string to the general `Quantity` object:

In [ ]:
distance = u.Quantity('1 lyr')

Check the availabe units with tab completion on the units module, `u.<TAB>`.

Quantities can be also created using lists and arrays:

In [ ]:
distances = [1, 3, 10] * u.lightyear
print(distances)

distances = np.array([1, 3, 10]) * u.lightyear
print(distances)

In [ ]:
distances.value

In [ ]:
np.mean( distances)

The quantity object has a value attribute, which is a plain `numpy.ndarray`:

In [ ]:
type(distances.value)

And a unit, which is represented by a `astropy.units.core.Unit` object:

In [ ]:
distances.unit

In [ ]:
type(distances.unit)

In [ ]:
type(3.)

A quantity behaves in many ways just like a `numpy.ndarray` with an attached unit.

In [ ]:
distances * 10

Many numpy functions will work as expected and return again a `Quantity` object:

In [ ]:
np.max(distances)

In [ ]:
np.mean(distances)

But there are exceptions, where the unit handling is not well defined, e.g. in `np.log` arguments have to be dimensionless, such as:

In [ ]:
#np.log(30 * u.MeV) # Will raise an UnitConversionError
np.log(30 * u.MeV / (1 * u.MeV))

In [ ]:
from astropy import constants as const

print(const.c.to('km / s'))

Probably the most useful method on the `Quantity` object is the `.to()` method which allows to convert a quantity to different units:

In [ ]:
distance.to('meter')

In [ ]:
distance.to(u.parsec)

Quantities can be combined with any arithmetical expression to derive other quantities, `astropy.units` will propagate
the units correctly:

In [ ]:
speed_of_light = distance / u.year
print(speed_of_light.to('km/s'))

In [ ]:
from astropy import constants as const

print(const.c.to('km / s'))

In [ ]:
print(const.c.to('cm / ns'))

Here is a [list of available constants](http://docs.astropy.org/en/stable/constants/#module-astropy.constants).

If you write a function you can make sure the input is given in the right units using the [astropy.units.quantity_input](http://docs.astropy.org/en/stable/api/astropy.units.quantity_input.html#astropy.units.quantity_input) decorator: 

### Exercises

- (*easy*) How long does the light travel from the sun to the earth in minutes? How long does the light travel from the Galactic center (assume a distance of 8 kpc) in years? 
- (*intermediate*) The tzar bomb was about 50 TNT. One TNT is about 4.2 Giga Joule. How many Giga Joule were released by the Tsar bomb? If we were to convert the human body to energy, how many Tsar bomb would that be equivalent to? Assume a human of 70 Kg.
- (*advanced*) Define a new unit called `"baro-meter"`, which is eqivalent to 25 cm and use it to measure the height of the empire state building (assume a height of 381 meters). Please read the [Astropy documentation on combining and defining units](http://docs.astropy.org/en/stable/units/combining_and_defining.html) for an example how to do this (For other ways to measure the height of a building using a barometer see [barometer question on Wikipedia](https://en.wikipedia.org/wiki/Barometer_question)...)


In [ ]:
#tnt = u.Unit('TNT', 4.18*u.GJ)
#zar = 50e6*tnt
#zar = u.Unit('zar' , 243*u.PJ)
#human_body = 70* u.kg * const.c**2
#human_body.to( zar)

# 2. Coordinates

[back to top](#Table-of-contents)

With the submodule [astropy.coordinates](http://docs.astropy.org/en/stable/coordinates/) Astropy provides a framework to handle sky positions in various coordinate systems and transformations between them.


The basic class to handle sky coordinates is [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html):

In [ ]:
from astropy.coordinates import SkyCoord

It can be created by passing a position angle for longitude and latitude and a keyword specifying a coordinate frame:

In [ ]:
position_crab = SkyCoord(83.63 * u.deg,  22.01 * u.deg, frame='icrs')
print(position_crab)

As for `Quantities` the instanciation with `lists`, `arrays` or even `Quantities` also works:

In [ ]:
positions = SkyCoord([345., 234.3] * u.deg,  [-0.1, 0.2] * u.deg, frame='galactic')

Alternatively the angles can be specified as string:

In [ ]:
position_crab = SkyCoord('5h34m31.97s', '22d0m52.10s', frame='icrs')

# or

position_crab = SkyCoord('5:34:31.97', '22:0:52.10',
                         unit=(u.hour, u.deg), frame='icrs')

In [ ]:
position_crab

In [ ]:
position_crab.separation( position_crab )

In [ ]:
position_gal_cen = SkyCoord(0 * u.deg,0 * u.deg,  frame='galactic')
position_crab.separation( position_gal_cen )

Where in the first case the unit doesn't have to specified because it is encoded in the string via `'hms'` and `'dms'`.

A very convenient way to get the coordinates of an individual object is qerying the [Sesame](http://cds.u-strasbg.fr/cgi-bin/Sesame) database with `SkyCoord.from_name()`:

In [ ]:
SkyCoord.from_name('Polaris')

To transform the coordinates to a different coordinate system we can use `SkyCoord.transform_to()`:

In [ ]:
pos_gal = position_crab.transform_to('galactic')
pos_gal

For convenience we can also directly use the `.galactic` or `.icrs` attributes:

In [ ]:
position_crab.galactic

In [ ]:
position_crab.icrs

To access the `longitude` and `latitude` angles individually: 

In [ ]:
position_crab.data.lon

In [ ]:
position_crab.data.lat

## 2.1 ALT / AZ coordinates

In various cirumstances, e.g. for planning observations, it can be usefull to transform a sky coordinate into a position in the horizontal coordinate system given a location on earth and a time

See:  https://en.wikipedia.org/wiki/Azimuth#/media/File:Azimuth-Altitude_schematic.svg

In [ ]:
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time

We define a location using [EarthLocation](http://docs.astropy.org/en/stable/api/astropy.coordinates.EarthLocation.html):

In [ ]:
Padova= EarthLocation(lat=45.406435 * u.deg, lon=11.876761 * u.deg)
print(Padova.geodetic)



And a time using the [Time](http://docs.astropy.org/en/stable/api/astropy.time.Time.html) object:

In [ ]:
now = Time.now()
print(now)

In [ ]:
now += 7 * u.hour

In [ ]:
now

Now we can define a horizontal coordinate system using the [AltAz]([docs.astropy.org/en/stable/api/astropy.coordinates.AltAz.html) class and use it to convert from the sky coordinate:

In [ ]:
position = SkyCoord.from_name('Polaris')

In [ ]:
altazPadova = AltAz(obstime=now, location=Padova)
altaz       = position.transform_to(altazPadova)
print(altaz)

### Exercises

- (*easy*) Define the sky coordinate for your favorite astronomical object and find the angular distance to the Crab Nebula as well as the Galactic center.
- (*expert*) Make a plot of the height above horizon vs.time for the crab position at the location of Padova in the next 24 hours. Mark the time range where it is visible. Would the Crab Nebula be visible tonight?

In [ ]:
from astropy.time import TimezoneInfo

In [ ]:
now = Time.now()
print(now)
time_array = now + np.arange(0,24,0.2) * u.h


EST = TimezoneInfo(utc_offset=2*u.hour)  # UTC+2


times     = []
altitudes = []
for t in time_array:
    altazPadova = AltAz(obstime=t, location=Padova)
    altaz       = position.transform_to(altazPadova)
    altitudes.append(altaz.alt.to(u.deg).value )
    
    t = t.to_datetime(timezone=EST)
    
    times.append( t.hour + t.minute/60 + t.second/3600)

# 3. Tables

[back to top](#Table-of-contents)

Astropy provides the [Table](http://docs.astropy.org/en/stable/api/astropy.io.votable.tree.Table.html) class in order to handle data tables.



## 3.1 Basics

Table objects can be created as shown in the following

In [ ]:
from astropy.table import Table

In [ ]:
table = Table()

We add columns to the table like we would add entries to a dictionary

In [ ]:
table['Source_Name'] = ['Crab', 'Sag A*', 'Cas A', 'Vela Junior']
table['GLON'] = [184.5575438, 0, 111.74169477, 266.25914205] * u.deg
table['GLAT'] = [-5.78427369, 0, -2.13544151, -1.21985818] * u.deg
table['Source_Class'] = ['pwn', 'unc', 'snr', 'snr']

By executing the following cell, we get a nicely formatted version of the table printed in the notebook:

In [ ]:
table

## 3.2 Accessing rows and columns

We have access to the defined columns. To check which ones are availbe you can use `Table.colnames`:

In [ ]:
table.colnames

And access individual columns just by their name:

In [ ]:
table['GLON']

And also a subset of columns:

In [ ]:
table[['Source_Name', 'GLON']]

Often, it is handy to get the column data as [astropy.units.Quantity](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity) using the `.quantity` property:

In [ ]:
table['GLON'].quantity

Rows can be accessed using numpy indexing:

In [ ]:
table[0:2]

Or by using a boolean numpy array for indexing:

In [ ]:
selection = table['Source_Name'] == 'Crab'
table[selection]

There is also a more sophisticated indexing scheme, which is explained [here](http://docs.astropy.org/en/stable/table/indexing.html), but not covered in this tutorial.

## 3.3 Reading / Writing tables to disk
Astropy tables can be serialized into many formats. For an overview see [here](http://docs.astropy.org/en/latest/io/unified.html#built-in-table-readers-writers). To write the table in FITS format we can use:

In [ ]:
table.write('example.fits', overwrite=True, format='fits')

In [ ]:
Table.read('example.fits')

## Exercises

-  Add columns with the `RA` and `DEC` coordinates of the objects to the example table.

# 4. Read FITS files

[back to top](#Table-of-contents)

The [flexible image transport system](https://fits.gsfc.nasa.gov/fits_documentation.html) format (FITS) is widely used data format for astronomical images and tables. As example we will use idata from the Crab nebula taken with the MAGIC telescope

See also: https://gamma-astro-data-formats.readthedocs.io/en/v0.3/general/time.html




In [ ]:
from astropy.io import fits

To open the fits file we use `fits.open()` and just specify the filename as an argument:

In [ ]:
# Get the value of the environment variable
import os

gammapy_data_path = os.environ.get('GAMMAPY_DATA')
if not gammapy_data_path:
    raise ValueError("The GAMMAPY_DATA environment variable is not set!")


In [ ]:
# Construct the full path to the FITS file
# 'cta-1dc/data/baseline/gps/gps_baseline_111630.fits'
# 'magic/rad_max/data/magic_dl3_run_05029748.fits'
fits_file_path = os.path.join(gammapy_data_path, '1.1/magic/rad_max/data/20131004_05029747_DL3_CrabNebula-W0.40+035.fits')
print(fits_file_path)

In [ ]:
fits_file = fits.open(fits_file_path)

We can retrieve some basic information on the  header data unit (HDU) by calling `.info()`:

In [ ]:
fits_file.info()

## 4.1 Primary

In [ ]:
primary = fits_file['PRIMARY'] 

#or

primary = fits_file[0] 

In [ ]:
primary.data

Additional meta information is stored in the `.header` attribute:

In [ ]:
primary.header

## 4.2 Events

In [ ]:
events = fits_file['EVENTS']

Using header we get all the information on how this events were collected

In [ ]:
events.header

In [ ]:
events.columns.names

In Astropy Table format

In [ ]:
events_table = Table( events.data )
events_table

In [ ]:
ra  = events_table['RA']
dec = events_table['DEC']

In [ ]:
# Direction of the events
fig, ax =  plt.subplots(figsize=(7,7))
#ax = plt.subplot(projection=wcs)

ax.scatter(ra,dec, s=0.2, alpha=0.5, label="All Events")



plt.ylabel('Dec [deg]')
plt.xlabel('RA [deg]')
plt.title('Sky Event Distribution')
ax.legend(loc="best")

In [ ]:
energies  = events_table['ENERGY']

In [ ]:

binning = np.logspace(-2,1,20)

plt.hist(energies,bins=binning,alpha=0.6,log=True)
plt.xscale('log')

plt.xlabel('Energy [TeV]')
plt.ylabel('Counts')
plt.title('Energy distribution')

## Excercise

 - Add on the Sky plot the poition of the source and the position of the pointing of the telescope

## 4.3 GTI

In [ ]:
gti = fits_file['GTI']

In [ ]:
gti.header

In [ ]:
Table( gti.data )

## 4.4 Effective Area

In [ ]:
effective_area = fits_file['EFFECTIVE AREA']

In [ ]:
effective_area.header

In [ ]:
Table( effective_area.data)

## 4.5 Energy Dispersion

In [ ]:
en_disp = fits_file['ENERGY DISPERSION']

In [ ]:
en_disp.header

In [ ]:
Table( en_disp.data )